### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

### Pegando os jogos do dia

In [2]:
# Obter a data de hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)

### Atualizando os datasets que tem jogo hoje

In [3]:
dia = ontem

In [4]:
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].tolist()
print(f'Jogos do dia {dia_jogos}\n')
results = atualiza_ultimos_jogos(id_jogos = id_jogos)
jogos_passados = jogos_passados.merge(results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
jogos_passados = jogos_passados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
jogos_passados.sort_values('Time', inplace=True)
jogos_passados.reset_index(inplace=True, drop=True)
display(jogos_passados[['Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']])

Jogos do dia 2023-10-20



The msedgedriver version (117.0.2045.31) detected in PATH at c:\Users\johnn\OneDrive\Documentos\apostas\basketball_models\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (118.0.2088.57); currently, msedgedriver 118.0.2088.57 is recommended for MicrosoftEdge 118.*, so it is advised to delete the driver in PATH and retry
100%|██████████| 10/10 [00:07<00:00,  1.31it/s]


,Home,Away,Home_Pts,Away_Pts,Status
0,Indiana Pacers,Cleveland Cavaliers,109.0,104.0,Encerrado
1,Philadelphia 76ers,Atlanta Hawks,120.0,106.0,Encerrado
2,Toronto Raptors,Washington Wizards,134.0,98.0,Encerrado
3,Dallas Mavericks,Detroit Pistons,114.0,104.0,Encerrado
4,Houston Rockets,Miami Heat,110.0,104.0,Encerrado
5,Milwaukee Bucks,Memphis Grizzlies,124.0,116.0,Encerrado
6,Argentino,Obras,76.0,86.0,Encerrado
7,Independiente de Oliva,San Martin,67.0,59.0,Encerrado
8,Golden State Warriors,San Antonio Spurs,117.0,122.0,Encerrado
9,Hiroshima D.,Utsunomiya Brex,83.0,81.0,Encerrado


In [6]:
jogos_passados = jogos_passados[(jogos_passados['Status'] == 'Após Tempo Extra') | (jogos_passados['Status'] == 'Encerrado') | (jogos_passados['Status'] == 'Após Pênaltis')]
restantes = len(results) - len(jogos_passados)
jogos_passados = jogos_passados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, jogos_passados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
n_depois = len(temporada)

temporada.to_csv('data/Entire_Season.csv', index=False)

n_atualizacao = n_depois - n_antes
print(f'Dataset de temporada atualizado com {n_atualizacao} jogos.\n')
print(f'Atenção! {restantes} jogos não finalizados.\n')

Dataset de temporada atualizado com 10 jogos.

Atenção! 0 jogos não finalizados.

